Nome: Pedro Paulo Dantas Silva Martins

Lista 2, respostas.

**1 - (Kernels em todo canto)**

a) Sabemos que o $\beta_0$ captura justamente o deslocamento dos nossos dados, caso os dados não fossem deslocados (com relação a origem), o nosso $\beta_0$ seria 0.<br>
E o ato de tirarmos a média de todos os dados, é justamente o processo que centraliza os dados com relação a origem. Logo quando fazemos isso o $\beta_0 = 0$.

b) $\hat{\beta} = argmin_{\tilde{\beta}} \sum_{i=1}^n (y_i - \sum_{j=1}^p x_{ij} \tilde{\beta_j})^2 + \lambda \sum_{j=1}^p \tilde{\beta_j}^2$<br>
É possível perceber que nesse caso não há uso de matrizes e vetores. A conta é feita pelas entradas. Transformarei para a forma vetorial e matricial.


$\hat{\beta} = argmin_{\tilde{\beta}} (Y - X \tilde{\beta})^T (Y - X \tilde{\beta}) + \lambda I \tilde{\beta}^T \tilde{\beta}$ Sabemos que o problema é convexo. Logo, se derivarmos em relação ao $\tilde{\beta}$ e igualarmos a 0 será o ponto de mínimo global, e portanto acharemos o nosso $\hat{\beta}$

$-2X^T (Y - X \hat{\beta}) + 2\lambda I \hat{\beta} = 0$ &harr; $-X^T Y + X^T X \hat{\beta} + \lambda I \hat{\beta} = 0$ &harr; $(X^T X + \lambda I
\hat{\beta} = X^T Y$ &harr; $\hat{\beta} = (X^T X + \lambda I)^{-1} X^T Y$

É possível perceber que realmente quando $\lambda = 0$ temos que o nosso $\hat{\beta}$ é igual ao de uma regressão linear.

c) $\hat{\beta} = (X^T X + \lambda I)^{-1} X^T Y$ Usando a fórmula sugerida pelo enunciado temos:<br>
$\hat{\beta} = (\frac{1}{\lambda} I - \frac{1}{\lambda} I X^T (I + X \frac{1}{\lambda} I X^T)^{-1}X \frac{1}{\lambda} I) X^T Y$<br>
$\hat{\beta} = \frac{1}{\lambda}IX^TY - \frac{1}{\lambda} I X^T(I + X\frac{1}{\lambda}I X^T)^{-1} X \frac{1}{\lambda} I X^T Y$ (Obs.: Perceba que $\frac{1}{\lambda} I X^T = X^T \frac{1}{\lambda} I$. Logo botamos o $X^T$ em evidência.)<br>
$\hat{\beta} = X^T(\frac{1}{\lambda} I Y - \frac{1}{\lambda} I (I + X \frac{1}{\lambda} I X^T)^{-1} X \frac{1}{\lambda} I X^T Y)$; Se definirmos nosso $\alpha = \frac{1}{\lambda} I Y - \frac{1}{\lambda} I (I + X \frac{1}{\lambda} I X^T)^{-1} X \frac{1}{\lambda} I X^T Y$<br>
$\hat{\beta} = X^T \alpha$ &harr; $\hat{\beta} = \sum_{i=1}^{n} \alpha_i x_i$

d) Sabemos que uma previsão é feita $y_* = \beta x_*$ ($\beta_0 = 0$). Portanto a previsão seria:<br>
$y_* = \sum_{i=1}^{n} \alpha_i x_i x_*$

e) Já constatamos que $f(x) = \sum_{i=1}^{n} \alpha_i x_i x$

É importante ressaltar a dimensão das coisas. $x_i \in \mathcal{R}^P$ são as observações anteriores sendo uma linha da Matriz principal. $x$ é um vetor com os parâmetros da nova observação. Portanto, é fácil ver que:<br>
$f(x) = \sum_{i=1}^{n} \alpha_i \langle x_i', x \rangle$

Agora podemos substituir esse produto interno por alguma outra função que respeite certas propriedades, e chamamos essa função de kernel $\mathbf{K}(.,.)$.

Logo podemos reescrever a $f(x)$ para um kernel qualquer:<br>
$f(x) = \sum_{i=1}^{n} \alpha_i \mathbf{K}(x_i',x)$

f) Realmente a fórmula para o coeficiente que achamos é a mesma para o SVM. Porém, lembrando das condições KKT para SVM, para todo ponto que não seja ponto de suporte o nosso $\alpha$ do ponto será $0$.

Logo uma diferença essencial é justamente a esparsidade do nosso $\alpha$, há muitos $0$ no SVM.

In [1]:
# G)
# Código pronto

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , KFold
from sklearn.linear_model import Ridge , LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from tqdm import tqdm


number_of_folds = 10
data = pd.read_csv("./bodyfat.csv")
X = data.drop(columns =["BodyFat","Density"])
y = data["BodyFat"]
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size =0.2, random_state =0)

In [2]:
# Fazendo a normalização dos dados.
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
# (i) MSE da regressão linear nos dados de teste:
lr = LinearRegression().fit(X_train, y_train)
y_hat_lr = lr.predict(X_test)

def MSE(y_hat, y=y_test):
    tamanho = y_hat.shape[0]
    MSE = np.sum((y_hat - y)**2)/tamanho
    return MSE

lr_MSE = MSE(y_hat_lr)
print(lr_MSE)

15.910489104854406


In [4]:
# (ii) MSE do ridge nos dados de teste:
from sklearn.preprocessing import StandardScaler

alphas = 10**np.linspace(3,-2,100)
ridge_cv_mse = []

for alpha in tqdm(alphas):
    cv_MSE = []
    folds = KFold(n_splits=number_of_folds, shuffle=True, random_state=2023).split(X_train , y_train)
    y_vector = np.array(y_train)
    for train_idx , val_idx in folds:
        ridge_pipeline = make_pipeline(StandardScaler(), Ridge(alpha=alpha))
        ridge_pipeline[1].fit(X_train[train_idx], y_vector[train_idx]) # treine seu modelo nos folds de treino
        y_hat = ridge_pipeline[1].predict(X_train[val_idx]) # faça suas previsões no fold de validação
        cv_MSE.append(MSE(y_hat, y_vector[val_idx])) # salve o MSE encontrado no fold de validação

    ridge_cv_mse.append(np.mean(cv_MSE))

index_min = ridge_cv_mse.index(min(ridge_cv_mse))
optimal_alpha = alphas[index_min] # encontre o melhor valor de alpha
ridge_pipeline = make_pipeline(StandardScaler(), Ridge(alpha=optimal_alpha))
ridge_pipeline[1].fit(X_train, y_train) # treine seu modelo em todo o conjunto de treino

y_hat_ridge = ridge_pipeline[1].predict(X_test)
ridge_MSE = MSE(y_hat_ridge)
print(ridge_MSE)

100%|██████████| 100/100 [00:01<00:00, 52.30it/s]

15.872514115940753


In [5]:
# (iii) e (IV) Obtendo os optimal hyperparams para a kernel ridge e também calculando seu MSE

kernels = ["linear", "polynomial", "rbf", "laplacian"]
gammas = [10**-3]
alphas = 10**np.linspace(3, -2, 100)
hyperparams = [( kernel , gamma , alpha) for kernel in kernels for gamma in gammas for alpha in alphas]
Kernelridge_cv_mse = []

for hyperparam in tqdm(hyperparams):
    kernel = hyperparam[0]
    gamma = hyperparam[1]
    alpha = hyperparam[2]

    cv_MSE = []
    folds = KFold(n_splits=number_of_folds, shuffle=True, random_state=2023).split(X_train , y_train)
    y_vector = np.array(y_train)
    for train_idx , val_idx in folds:
        Kernelridge_pipeline = make_pipeline(StandardScaler(), KernelRidge(alpha=alpha, kernel=kernel, gamma=gamma))
        Kernelridge_pipeline[1].fit(X_train[train_idx], y_vector[train_idx]) # treine seu modelo nos folds de treino
        y_hat = Kernelridge_pipeline[1].predict(X_train[val_idx]) # faça suas previsões no fold de validação
        cv_MSE.append(MSE(y_hat, y_vector[val_idx])) # salve o MSE encontrado no fold de validação

    Kernelridge_cv_mse.append(np.mean(cv_MSE))

index_min = Kernelridge_cv_mse.index(min(Kernelridge_cv_mse))
optimal_hyperparam = hyperparams[index_min] # encontre o melhor valor de alpha
optimal_kernel = optimal_hyperparam[0]
optimal_gamma = optimal_hyperparam[1]
optimal_alpha = optimal_hyperparam[2]

Kernelridge_pipeline = make_pipeline(StandardScaler(), KernelRidge(alpha=optimal_alpha, kernel=optimal_kernel, gamma=optimal_gamma))
Kernelridge_pipeline[1].fit(X_train, y_train) # treine seu modelo em todo o conjunto de treino

y_hat_Kernelridge = Kernelridge_pipeline[1].predict(X_test)
Kernelridge_MSE = MSE(y_hat_Kernelridge)
print(Kernelridge_MSE)
print(optimal_hyperparam)

100%|██████████| 400/400 [00:13<00:00, 30.27it/s]


14.663798962219209
('rbf', 0.001, 0.01)
